# OpenAI Function Calling


In [7]:
import os
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=os.environ['OPENAI_API_KEY']
)


In [8]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [9]:
# define a function
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

In [10]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

In [11]:
# Without functions
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages
)

In [13]:
response

ChatCompletion(id='chatcmpl-9MzvMa7lh4rCmRKT4Ql5Cf97UzAlz', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I am an AI and do not have real-time information. Please check a weather website or app for the current weather in Boston.', role='assistant', function_call=None, tool_calls=None))], created=1715268280, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=26, prompt_tokens=15, total_tokens=41))

In [14]:
response.choices[0].message.content.strip()

'I am an AI and do not have real-time information. Please check a weather website or app for the current weather in Boston.'

In [20]:
# With functions
# Note: Function calling with openAi doesn't directly call the function
# we still have to do that ourselves
# it just tells us what function to call, the arguments to that function
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions
)

In [25]:
response

ChatCompletion(id='chatcmpl-9MzyW2bFltYWABYOb3EHCedPyoBuN', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None))], created=1715268476, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=18, prompt_tokens=82, total_tokens=100))

In [28]:
response_message = response.choices[0].message
response_message

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None)

In [29]:
response_message.content

In [30]:
response_message.function_call

FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather')

In [34]:
args = json.loads(response_message.function_call.arguments)
args

{'location': 'Boston, MA'}

In [35]:
get_current_weather(**args)

'{"location": "Boston, MA", "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

In [36]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]

In [37]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions
)

In [41]:
response.choices[0]

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I help you today?', role='assistant', function_call=None, tool_calls=None))

In [39]:
response.choices[0].message.content

'Hello! How can I help you today?'

In [53]:
messages1 = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

# Note: function_call parameter is set to "auto" by default
# In this mode, the model will decide whether to call a function or not

# There are two other modes of function_call: {"name": "get_current_weather"} and "none"
# First one forces llm to always call a specific function, second one never calls a function

response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    # function_call="auto"
    function_call={"name": "get_current_weather"}
    # function_call="none"
)

In [54]:
messages.append(response.choices[0].message)

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None)

In [55]:
args = json.loads(response.choices[0].message.function_call.arguments)
observation = get_current_weather(**args)

In [56]:
messages.append({
    "role": "function",
    "name": "get_current_weather",
    "content": observation,
})

In [57]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
)

In [59]:
response

ChatCompletion(id='chatcmpl-9N0VIS6AeyGZwKeKzKscI36LEd7hb', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The weather in Boston is currently sunny and windy with a temperature of 72°F.', role='assistant', function_call=None, tool_calls=None))], created=1715270508, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=17, prompt_tokens=53, total_tokens=70))

In [58]:
response.choices[0].message.content

'The weather in Boston is currently sunny and windy with a temperature of 72°F.'